<a href="https://colab.research.google.com/github/cedro3/BlendGAN/blob/main/BlendGAN_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### セットアップ

In [ ]:
#@title セットアップ

# githubからコードを取得
! git clone https://github.com/cedro3/BlendGAN.git
%cd BlendGAN

# ninjaインストール
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

# 学習済みパラメータのダウンロード
import gdown
gdown.download('https://drive.google.com/uc?id=1D27HPNOSx9kWIhc13VevRy0pUv_xYiJb', './pretrained_models/blendgan.pt', quiet=False)
gdown.download('https://drive.google.com/uc?id=1pWWSm_c75ieMExJPWJuYA1wby-hm4f1J', './pretrained_models/psp_encoder.pt', quiet=False)
gdown.download('https://drive.google.com/uc?id=1qshfqj8SdmgQv_kfLpiohbI3QPQF-OE5', './pretrained_models/style_encoder.pt', quiet=False)

# ランドマークデータのダウンロード
! wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
! bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

# 画像の表示関数
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np

def display_pic(folder):
    fig = plt.figure(figsize=(30, 40))
    files = os.listdir(folder)
    files.sort()
    for i, file in enumerate(files):
        img = Image.open(folder+'/'+file)    
        images = np.asarray(img)
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images)
        ax.imshow(image_plt)
        ax.set_xlabel(folder+'/'+file, fontsize=15)               
    plt.show()
    plt.close()


### デモ

In [ ]:
#@title Style tranfer デモ

# style_transferフォルダーリセット
import os
import shutil
if os.path.isdir('results/style_transfer'):
    shutil.rmtree('results/style_transfer')

# style_transfer実行
! python style_transfer_folder.py --size 1024 --ckpt ./pretrained_models/blendgan.pt --psp_encoder_ckpt ./pretrained_models/psp_encoder.pt --style_img_path ./test_imgs/style_imgs/ --input_img_path ./test_imgs/face_imgs/ --outdir results/style_transfer/

# imagesフォルダーリセット
if os.path.isdir('results/images'):
    shutil.rmtree('results/images')
os.makedirs('results/images', exist_ok=True)

# output.mp4をリセット
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

# 画像の連結
import glob
img1 = glob.glob('results/style_transfer/01_*.jpg')
img2 = glob.glob('results/style_transfer/02_*.jpg')
img3 = glob.glob('results/style_transfer/03_*.jpg')
img1.sort()
img2.sort()
img3.sort()

import cv2
for i in range(len(img1)):
    im1 = cv2.imread(img1[i])
    im2 = cv2.imread(img2[i])
    im3 = cv2.imread(img3[i])
    tmp = cv2.vconcat([im1, im2])
    out = cv2.vconcat([tmp,im3])
    all = cv2.resize(out, dsize=(1024, 1024))
    cv2.imwrite('results/images/'+str(i).zfill(3)+'.jpg', all)

# 画像を動画に変換
!ffmpeg -r 0.6 -i results/images/%03d.jpg -vcodec libx264 -pix_fmt yuv420p output.mp4

In [ ]:
#@title 動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="70%" height="70%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

### オリジナル画像でやってみる

In [ ]:
#@title picフォルダーリセット
import os
import shutil
if os.path.isdir('pic'):
     shutil.rmtree('pic')
os.makedirs('pic', exist_ok=True)

**必ずpicフォルダーに画像をアップロードしてから下記を実行**

In [ ]:
#@title 顔画像の切り出し
import os
import shutil
from tqdm import tqdm

if os.path.isdir('align'):
     shutil.rmtree('align')
os.makedirs('align', exist_ok=True)

def run_alignment(image_path):
  import dlib
  from alignment import align_face
  predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
  aligned_image = align_face(filepath=image_path, predictor=predictor) 
  return aligned_image 

path = './pic'
files = sorted(os.listdir(path))
for i, file in enumerate(tqdm(files)):
  if file=='.ipynb_checkpoints':
     continue
  input_image = run_alignment(path+'/'+file)
  input_image.resize((1024,1024))
  input_image.save('./align/'+file)

display_pic('align')

In [ ]:
#@title 画像ファイル指定
input = ""#@param {type:"string"}
file = './align/'+input

# original_imagesフォルダーリセット
if os.path.isdir('test_imgs/original_imgs'):
    shutil.rmtree('test_imgs/original_imgs')
os.makedirs('test_imgs/original_imgs', exist_ok=True)

# original_imagesフォルダーへコピー
import shutil
shutil.copy(file, 'test_imgs/original_imgs/'+input) 


In [ ]:
#@title Style transfer を実行し動画を作成

# style_transferフォルダーリセット
if os.path.isdir('results/style_transfer'):
    shutil.rmtree('results/style_transfer')

! python style_transfer_folder.py --size 1024 --ckpt ./pretrained_models/blendgan.pt --psp_encoder_ckpt ./pretrained_models/psp_encoder.pt --style_img_path ./test_imgs/style_imgs/ --input_img_path ./test_imgs/original_imgs/ --outdir results/style_transfer/

# imagesフォルダーリセット
import os
import shutil
if os.path.isdir('results/images'):
    shutil.rmtree('results/images')
os.makedirs('results/images', exist_ok=True)

# output.mp4リセット
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

# 画像のリサイズ
import cv2
import glob
files = glob.glob('results/style_transfer/*.jpg')
files.sort()
for i, file in enumerate(files):
    img = cv2.imread(file)
    img_resize = cv2.resize(img, dsize=(1536, 512))
    cv2.imwrite('results/images/'+str(i).zfill(3)+'.jpg', img_resize)

# 画像を動画に変換
!ffmpeg -r 0.6 -i results/images/%3d.jpg -vcodec libx264 -pix_fmt yuv420p output.mp4

In [ ]:
#@title 動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="100%" height="100%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")